# Training the Change Detection Model (ResNet10)

In [9]:
import os
import re
import cv2
import glob
import json
import time
import random
import string
import imageio
import collections
import numpy as np
from PIL import Image
from typing import Tuple
from shapely.geometry import Polygon
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from models.changedetection_model import ChangeDetectionModelResNet101
from dataset.syntheticpairs_dataset import SyntheticPairsDataSet

In [10]:
model_save_path = ""

In [11]:
annotation_path = "C:\\Users\\Ugne\\Documents\\studies\\Python\\DLGroupTask\\synthetic_anno.json"
json_file = open(annotation_path)
anno = json.load(json_file)
json_file.close()

In [12]:
available_scenes = [im["scene"] for im in anno["images"]]
available_scenes_df = pd.DataFrame(available_scenes, columns=["scene_id"])

len(available_scenes)

500

## Training

In [13]:
# This dictionary defines hyperparameters for the dataset
# Complete configurations for each experiment can be found in the configs folder
train_set_params = {
        "dataset_name": "syntheticpairs",
        "root": "D:\\DeepLearningFiles\\renders_multicam_diff_all",
        "loader_params": {
            "batch_size": 8,
            "drop_last": False,
            "pin_memory": True,
            "num_workers": 8
        },
        "mode": "train",
        "crop": True,
        "resize": True,
        "spatial_resolution": [256, 180],
        "overfit": False,
        "normalize": True,
        "augment": True,
        "shuffle": True
    }

trainset = SyntheticPairsDataSet(train_set_params)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_set_params["loader_params"]["batch_size"],
                                          shuffle=train_set_params["shuffle"], num_workers=train_set_params["loader_params"]["num_workers"])



In [20]:
def get_unique_label_values(idx):
    img1, img2, label, folder, depth1, depth2 = trainset.__getitem__(idx)
    return np.unique(np.array(label.type(torch.LongTensor)))

unique_vals = [get_unique_label_values(idx) for idx in range(len(trainset))]

In [21]:
np.unique(np.concatenate(unique_vals))

array([0, 2, 3], dtype=int64)

In [14]:
# Print number of batches
len(trainloader)

50

In [15]:
model_params = dict({
        "model_name": "changedetection",
        "num_classes": 4,
        "dataset_name": "syntheticpairs",
        "rgb": True, # Use RGB inputs
        "depth": False, # Use depth inputs
        "spatial_resolution": [256, 180],
        "mode": "train",
        "max_epochs": 20, # originally set to 4000
        "lr": 0.0004,
        "save_path": "/app/saved_models/changedetection_benchmark/",
        "weights_path": "/app/saved_models/changedetection_benchmark/",
        "load_weights": -1,
        "val_epochs": [1950],
        "val_rate": 50,
        "save_rate": 50,
        "lr_policy": "cosine",
        "lr_decay_iters": 10,
        "batch_size": 16,
        "loss_weights": True
    })
model = ChangeDetectionModelResNet101(model_params).model

Loaded weights into backbone
Number total params  39643962
Loaded weights into backbone
Number total params  58636090


In [16]:
# Print model
model

SingleStreamResNet101(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [17]:
# Parallelize model across GPUs if available
if torch.cuda.is_available():
    model = model.cuda()
    model = torch.nn.DataParallel(model)

# Loss definitions
loss_weights = torch.Tensor([0.0035252888617000786, 0.33161259399877635, 0.3316452266650099, 0.3332168904745137]).cuda()
criterion_loss = torch.nn.CrossEntropyLoss(weight=loss_weights) # Segmentation output loss

# Optimizer definitions
optimizer = torch.optim.SGD(model.parameters(), lr=model_params["lr"])

In [18]:
# Main training loop over epochs
starting_epoch = 0
num_epochs = model_params['max_epochs']
for epoch in range(starting_epoch, num_epochs):
    train_iterations = len(trainloader)
    train_batch_size = model_params['batch_size']
    
    model.train()
    running_loss = 0
    # Training loop over samples within epoch
    for i, data in enumerate(trainloader):
        # Process data to correct types
        img1, img2, label, scene, depth1, depth2 = data
        img1 = img1.float()
        img2 = img2.float()
        depth1 = depth1.float()
        depth2 = depth2.float()
        label = label.type(torch.LongTensor)
        
        # Move data to GPU if available
        if torch.cuda.is_available():
            img1 = img1.cuda()
            img2 = img2.cuda()
            depth1 = depth1.cuda()
            depth2 = depth2.cuda()
            label = label.cuda()
         
        # Get model output
        model_input = (img1, img2, depth1, depth2)
        model_out = model(model_input)
        
        # Calculate loss function, get gradients, and update network weights
        loss = criterion_loss(model_out, label)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()

    print("Epoch {} Loss: ".format(epoch), running_loss/len(trainloader))

Epoch 0 Loss:  1.357935118675232
Epoch 1 Loss:  1.209993646144867
Epoch 2 Loss:  1.1167628419399263
Epoch 3 Loss:  1.032520158290863
Epoch 4 Loss:  0.9585029625892639
Epoch 5 Loss:  0.9038837468624115
Epoch 6 Loss:  0.9504693007469177
Epoch 7 Loss:  0.8961683368682861
Epoch 8 Loss:  0.8636924231052399
Epoch 9 Loss:  0.8487482225894928
Epoch 10 Loss:  0.7886814713478089
Epoch 11 Loss:  0.8040128910541534
Epoch 12 Loss:  0.7631347894668579
Epoch 13 Loss:  0.8123396903276443
Epoch 14 Loss:  0.736968287229538
Epoch 15 Loss:  0.7108311611413956
Epoch 16 Loss:  0.7481522595882416
Epoch 17 Loss:  0.7670660638809204
Epoch 18 Loss:  0.7399828797578811
Epoch 19 Loss:  0.6862278836965561


In [19]:
# Export trained model
from datetime import datetime
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S").replace(":","").replace(" ","").replace("-","")

torch.save(model.state_dict(), f'change_detection_model_resnet101_{current_time}.pt')